In [1]:
import numpy as np
import copy
import pickle

from scipy import ndimage
from BaselineRemoval import BaselineRemoval
import matplotlib.pyplot as plt
import os
from scipy import interpolate
from scipy import signal
from scipy.fft import fft, ifft, fftfreq
from sklearn.decomposition import PCA
import timeit
from os import path

plt.rcParams['figure.figsize'] = (20.0, 10.0)
# plt.rcParams['figure.figsize'] = (10.0, 5.0)
plt.rcParams['figure.dpi'] = 500
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

Mogelijke verbeteringen:
 - maak de covariance afhankelijke van de noise 
 - draai de spectrum zodanig dat de globale sloop de x-as is. Zodat het min-max algorithme beter werkt. DONE
 - de smoothing die applied wordt door de covariantie moet afgesteld worden per sample

Problemen:
 - initializatie maakt uit voor de kalman smoother (hoeveel noise er is vs raman)
 - aantal iteratie maakt uit voor de kalman smoother (hoeveel noise er is vs raman)
 - de bounderies hebben geen invloed op de kalman smoother buiten alles binnen de bounderies houden.
   Tenzij smoothing ook aanstaat.
   
   
excitation laser maakt uit hoe de raman laser eruit ziet. Hoger (rood) betekend lower raman signal en een breder signal

date: 2-9-2022

 - stabilise the logic part with neighbourhood information
 - where the logic part is true make the line segments bigger (does not work)
 - find something to deterime the smoothness of the poly fit (no bumbs) DONE


### Ideas:

- Wavelet transform

### Errors:

 - smooth_grad fix the try except clause such that and the beginning and end of the spectrum it also works
 - using fourier transform it seems that gibbs phenomena is a problem


In [2]:
file_location = "../data/Raman_Mouse/corrected_4_wavenumbers/"
# file_location = "../data/Green_excitation/corrected_4_wavenumbers/"

filenames = np.load(f"{file_location}FileNames.npy")
with open(f"{'/'.join(file_location.split('/')[:-2])}/Sample_labels.pickle", 'rb') as f:
    labels = pickle.load(f)
labels = {l.split(".")[0]: val for l, val in labels.items()}

wavenumbers = np.load(f"{file_location}Wavenumbers.npy")
    
data = []
for f in filenames:
    x = np.load(f"{file_location}{f.split('.')[0]}.npy")
    data.append(((x.reshape(-1,x.shape[-1])).reshape(x.shape), labels[f.split(".")[0]]))
    
# #mouse image
# noise_amount = 1e5
# intervals = 10
# general_noise = 10e6
# poly_fit = 1e5
# precision = 12
# continue_gap = 100
# # green image
# noise_amount = 3e5
# general_noise = 5e6
# intervals = 30
# precision = 20
# poly_fit = 8e5
# continue_gap = 400

In [3]:
def remove_noise_fft_min(min_HWHM=5, spline_appr=5e9):
    k = int((wavenumbers[-1] - wavenumbers[0]) / (2*min_HWHM))
    def remove_noise_fft(x):       
        x_ = copy.copy(x)
        mean = np.mean(x)
        x_ -= mean
        fourier = fft(x_)

        fourier_real = fourier.real
        fourier_imag = fourier.imag
        
        space = np.arange(x.shape[0])

        func = interpolate.UnivariateSpline(space, fourier_real, k=4, s=spline_appr)
        fourier_real[k:-k] = func(space[k:-k])

        func = interpolate.UnivariateSpline(space, fourier_imag, k=4, s=spline_appr)
        fourier_imag[k:-k] = func(space[k:-k])

        smooth_fourier = fourier_real + 1j * fourier_imag

        return ifft(smooth_fourier) + mean
    return remove_noise_fft
remove_noise_fft = remove_noise_fft_min()

In [4]:
def gaussian(x, mu, sigma, scale):
    x = scale * np.exp(-0.5* ((x - mu) / sigma)**2)
    return x * (x > min(scale/10, 10))

def init_fit_gaussian(min_wavenumber=-1000, left_steps=10, min_sigma=100, max_sigma=1500, steps=20):
    def fit_gaussian(x):
        lst = []
        for mu in np.concatenate((np.linspace(-min_wavenumber, 0, left_steps), np.linspace(wavenumbers[0], wavenumbers[-1], steps))):
            temp_mu = 0,1,1
            temp_intergral = 0
            for sigma in np.linspace(min_sigma**0.5, max_sigma**0.5, steps)**2:
                if mu < 0 and mu + 3*sigma < 0:
                    continue

                scale = 10
                current = gaussian(wavenumbers, mu, sigma, scale)

                for delta in 10**np.arange(10, -1, -1):
                    while not np.sum((current - x) > 0):
                        scale += delta
                        current = gaussian(wavenumbers, mu, sigma, scale)
                    else:
                        scale -= delta
                        current = gaussian(wavenumbers, mu, sigma, scale)

                intergral = np.sum(current)
                if intergral > temp_intergral:
                    temp_intergral = intergral
                    temp_mu = mu, sigma, scale

            if temp_mu[1] < min_sigma: #no fit found
                continue

            for sigma in np.linspace(max(min_sigma, temp_mu[1]-50), temp_mu[1]+50, steps):
                if mu < 0 and mu + 3*sigma < 0:
                    continue
                scale = 10
                current = gaussian(wavenumbers, mu, sigma, scale)

                for delta in 10**np.arange(10, -1, -1):
                    while not np.sum((current - x) > 0):
                        scale += delta
                        current = gaussian(wavenumbers, mu, sigma, scale)
                    else:
                        scale -= delta
                        current = gaussian(wavenumbers, mu, sigma, scale)

                intergral = np.sum(current)
                if intergral > temp_intergral:
                    temp_intergral = intergral
                    temp_mu = mu, sigma, scale

            lst.append(gaussian(wavenumbers, *temp_mu))
        lst = np.array(lst)
        return np.max(lst, axis=0)
    return fit_gaussian
    
def preliminary_photo_approximation(x):
    fit_gaussian = init_fit_gaussian()
    fit_gaussian2 = init_fit_gaussian(left_steps=0)
    
    x_ = copy.copy(x)
    gaussians = []   
    for fit in [fit_gaussian, fit_gaussian2]:
        current = fit(x_)
        x_ -= current
        gaussians.append(current)
    gaussians = np.array(gaussians)
    return np.sum(gaussians, axis=0)

#faster approximation
def preliminary_photo_approximation(x):
    spline_appr = 5e9
    ifft_appr = 1e6
    space = np.arange(x.shape[0])

    lst = []
    for min_HWHM in [100,150,200,250]:
        k = int((wavenumbers[-1] - wavenumbers[0]) / (2*min_HWHM))

        fourier = fft(x)
        fourier_real = fourier.real
        fourier_imag = fourier.imag

        func = interpolate.UnivariateSpline(space, fourier_real, k=4, s=spline_appr)
        fourier_real[k:-k] = func(space[k:-k])

        func = interpolate.UnivariateSpline(space, fourier_imag, k=4, s=spline_appr)
        fourier_imag[k:-k] = func(space[k:-k])

        smooth_fourier = fourier_real + 1j * fourier_imag
        lst.append(ifft(smooth_fourier))

    lst = np.min(np.array(lst), 0)
    fourier = fft(lst)
    fourier_real = fourier.real
    fourier_imag = fourier.imag

    func = interpolate.UnivariateSpline(space, fourier_real, k=4, s=spline_appr)
    fourier_real[k:-k] = func(space[k:-k])

    func = interpolate.UnivariateSpline(space, fourier_imag, k=4, s=spline_appr)
    fourier_imag[k:-k] = func(space[k:-k])

    smooth_fourier = fourier_real + 1j * fourier_imag
    x = ifft(smooth_fourier)
    
    func = interpolate.UnivariateSpline(space, x, k=4, s=ifft_appr)
    return func(space)


In [13]:
def zero_bound_to_None(bound):
    # -0 must be translate to None
    return -bound if bound != 0 else None

def init_smooth_grad(general_noise=10e6, continue_gap=100):
    def smooth_grad(poly2):
        general_grad_func = interpolate.UnivariateSpline(wavenumbers, 
                                                         poly2,
                                                         k=3, s=general_noise)
        general_grad = general_grad_func(wavenumbers)
        grad_general = general_grad[1:] - general_grad[:-1]

        grad = poly2[1:] - poly2[:-1]        
        grad2 = np.pad(poly2[2:] - 2 * poly2[1:-1] + poly2[:-2],(1, 0), 'edge')

        general_max_lst = signal.argrelmax(grad_general)[0]
        max_lst = signal.argrelmax(grad)[0]
        max_grad_lst = signal.argrelmax(grad2)[0]
        min_grad_lst = signal.argrelmin(grad2)[0]

        for max_ in max_lst:
            if general_max_lst is []:
                general_max = general_max_lst[np.argmin([abs(x - max_) for x in general_max_lst])]
                if abs(general_max - max_) < continue_gap/2:
                    continue

            try:
                *_, left = filter(lambda x: x < max_, max_grad_lst)
                right = next(filter(lambda x: x > max_, min_grad_lst))
            except (ValueError, StopIteration) as error:
                continue

            try:
                *_, left_max = filter(lambda x: x < max_, max_lst)
            except ValueError:
                left_max = 0

            try:
                right_max = next(filter(lambda x: x > max_, max_lst))
            except StopIteration:
                right_max = grad.shape[0]          

            old_sum = sum(grad)
            restore_grad = copy.copy(grad)
            grad[left:max_] = grad[left]
            new_sum = sum(grad)
            i = max_

            dist = 3 * min(max_ - left, right - max_)
            dist = min(dist, max_ - left_max, right_max - max_)

            while True:
                while new_sum < old_sum:
                    grad[i:i+10] = grad[left]
                    new_sum = sum(grad)
                    i += 10
                    if i > min(max_ + dist, grad.shape[0]):
                        left = min(left+10, grad.shape[0]-1) #to prevent left going out of bounds
                        grad = copy.copy(restore_grad)
                        grad[left:max_] = grad[left]
                        new_sum = sum(grad)
                        i = max_
                        break
                else:
                    break

            flat_residu = (old_sum - new_sum) / (left-i)
            grad[left:i] -= flat_residu

            # if flatting of the maximum gradient is not possible skip
            if left >= max_:
                continue

            smooth_dist = max(2, min(i-left, continue_gap))
            sigma = smooth_dist/4
            smooth_dist2 = smooth_dist//2
            smooth_dist2b = smooth_dist - smooth_dist2
            smooth_dist3 = smooth_dist + smooth_dist2
            if left-smooth_dist3 < 0: # correct for maximums at the left part of the spectrum
                left_bound, gaus_left_bound = max(0, left-smooth_dist2), max(0, left-smooth_dist)
                grad[left_bound:i+smooth_dist2] = ndimage.gaussian_filter(grad[gaus_left_bound:i+smooth_dist], sigma, mode='nearest')[left_bound - gaus_left_bound:-smooth_dist2b]
                left_bound = max(0, left-smooth_dist)
                grad[left_bound: left+smooth_dist] = ndimage.gaussian_filter(grad[:left+smooth_dist3], sigma, mode='nearest')[left_bound:-smooth_dist2]
                left_bound, gaus_left_bound = max(0, i-smooth_dist), max(0, i-smooth_dist3)
                grad[left_bound:i+smooth_dist] = ndimage.gaussian_filter(grad[gaus_left_bound:i+smooth_dist3], sigma, mode='nearest')[left_bound - gaus_left_bound:-smooth_dist2]
            elif i + smooth_dist3 > grad.shape[0]: # correct for maximums at the right part of the spectrum
                right_bound, gaus_right_bound = max(grad.shape[0] - (i+smooth_dist2), 0), max(grad.shape[0] - (i+smooth_dist), 0)
                right_diff = right_bound - gaus_right_bound
                right_bound, gaus_right_bound, right_diff = zero_bound_to_None(right_bound), zero_bound_to_None(gaus_right_bound), zero_bound_to_None(right_diff)
                grad[left-smooth_dist2:right_bound] = ndimage.gaussian_filter(grad[left-smooth_dist:gaus_right_bound], sigma, mode='nearest')[smooth_dist2b:right_diff]
                right_bound, gaus_right_bound = max(grad.shape[0] - (left+smooth_dist), 0), max(grad.shape[0] - (left+smooth_dist3), 0)
                right_diff = right_bound - gaus_right_bound
                right_bound, gaus_right_bound, right_diff = zero_bound_to_None(right_bound), zero_bound_to_None(gaus_right_bound), zero_bound_to_None(right_diff)
                grad[left-smooth_dist:right_bound] = ndimage.gaussian_filter(grad[left-smooth_dist3:min(left+smooth_dist3, grad.shape[0])], sigma, mode='nearest')[smooth_dist2:right_diff]
                right_bound = max(grad.shape[0] - (i+smooth_dist), 0)
                right_bound = zero_bound_to_None(right_bound)
                grad[i-smooth_dist:right_bound] = ndimage.gaussian_filter(grad[i-smooth_dist3:], sigma, mode='nearest')[smooth_dist2:right_bound]
            else:
                grad[left-smooth_dist2:i+smooth_dist2] = ndimage.gaussian_filter(grad[left-smooth_dist:i+smooth_dist], sigma, mode='nearest')[smooth_dist2b:-smooth_dist2b]            
                grad[left-smooth_dist:left+smooth_dist] = ndimage.gaussian_filter(grad[left-smooth_dist3:left+smooth_dist3], sigma, mode='nearest')[smooth_dist2:-smooth_dist2]
                grad[i-smooth_dist:i+smooth_dist] = ndimage.gaussian_filter(grad[i-smooth_dist3:i+smooth_dist3], sigma, mode='nearest')[smooth_dist2:-smooth_dist2]

            new_sum = sum(grad)
            left_bound, right_bound = max(0,left-smooth_dist), min(i+smooth_dist, grad.shape[0])
            flat_residu = (old_sum - new_sum) / (left_bound - right_bound)
            grad[left_bound: right_bound] -= flat_residu

        value = poly2[0]
        poly2 = [value]
        for g in grad:
            value += g
            poly2.append(value)
            
        return np.array(poly2)
    return smooth_grad

smooth_grad = init_smooth_grad()


In [14]:
def init_poly_approximation(general_noise=50e6, continue_gap=100, intervals=10, precision=12, poly_fit=1e5):
    def poly_approximation(x, general):
        general_grad_func = interpolate.UnivariateSpline(wavenumbers, 
                                                         general,
                                                         k=4, s=general_noise)
        general_grad = general_grad_func(wavenumbers)
        sec_grad2 = np.pad(general_grad[2:] - 2*general_grad[1:-1] + general_grad[:-2], (1, 1), 'edge')
        grad = np.pad(general[1:] - general[:-1], (0,1), 'edge')
        grad_m = np.mean(np.abs(grad))
        small_grad = np.abs(grad) < grad_m
        sec_grad_pos = (sec_grad2 < 0)

        small_grad = ndimage.minimum_filter(small_grad, size=continue_gap)
        sec_grad_pos = ndimage.minimum_filter(sec_grad_pos, size=continue_gap)
        small_grad = ndimage.maximum_filter(small_grad, size=continue_gap)
        sec_grad_pos = ndimage.maximum_filter(sec_grad_pos, size=continue_gap)
        logic = sec_grad_pos * small_grad

        sec_grad_sign = ndimage.gaussian_filter(logic.astype(np.float32), 50)       

        minimums = []
        step = x.shape[0]//intervals
        for j in range(0, step, step//precision):
            x_ = copy.copy(x)
            # to adress exponential values at the left side of the spectrum, make these value higher
            if j > 0:
                new_step = x_[:j].shape[0]
                slope = general[:j]
                height = slope[-1] - slope[0]
                slope = np.linspace(0, height, new_step)
                index = np.argmin(x_[:j] - slope)
                slope -= slope[index]
                slope += x[index]
                x_[:j] = slope

            for i in range(j, x.shape[0]-1, step):
                step_size = step
                new_step = x_[i:i+step_size].shape[0]
                smooth_curve = general[i:i+new_step]
                height = smooth_curve[-1] - smooth_curve[0]
                slope = np.linspace(0, height, new_step)
                index = np.argmin(x_[i:i+new_step] - slope)
                slope -= slope[index]
                slope += x[i+index]
                x_[i:i+new_step] = slope
            minimums.append(x_)

        poly_max = np.max(np.array(minimums), axis=0)
        poly_min = np.min(np.array(minimums), axis=0)
        poly = sec_grad_sign * poly_min + (1-sec_grad_sign) * poly_max

        old_poly = poly-1
        j = 0
        while sum(poly-old_poly) and j < 5:
            j += 1
            func = interpolate.UnivariateSpline(wavenumbers, 
                                                poly,
                                                k=4, s=poly_fit)
            poly2 = func(wavenumbers)

            old_poly = copy.copy(poly)
            """
            This pushes the poly graph down acting like a weight for the spline to fit the graph below x
            """
            problem_part = ndimage.maximum_filter(ndimage.minimum_filter(poly2 > x, 3), continue_gap//5)
            poly -= problem_part * 5
        poly2 = smooth_grad(poly2)

        return poly2
    return poly_approximation
poly_approximation = init_poly_approximation()

In [15]:
def split_signal(img):
    """ 
    img consists of data and label
    
    artefact due to using previous points 
    
    """

    x = img[0].reshape(-1, img[0].shape[-1])
    
    raman = np.empty(x.shape)
    photo = np.empty(x.shape)
    start = timeit.default_timer()
    for pixel in range(x.shape[0]):
        # poly approximation
        poly = preliminary_photo_approximation(x[pixel])
        poly2 = smooth_grad(poly)
        for i in range(3):
            poly2 = poly_approximation(x[pixel], poly2)

        # raman approximation
        raman2 = remove_noise_fft(x[pixel]-poly2)
        
        raman[pixel] = raman2
        photo[pixel] = poly2
        if not pixel % 100:
            stop = timeit.default_timer()
            print(f"progress = {100* pixel / x.shape[0]}%, 100 samples time : {stop - start}")
            start = timeit.default_timer()
        
    return raman, photo

In [17]:
file_location2 = f"{'/'.join(file_location.split('/')[:-2])}/approximated_curve_fit/"
os.makedirs(file_location2, exist_ok=True)

shape = data[0][0].shape

for j, (f, img) in enumerate(zip(filenames, data)):   
    start = timeit.default_timer()
    if path.exists(f'{file_location2}{f.split("/")[-1].split(".")[0]}_raman.npy'):
        print(f'file already exist {file_location2}{f.split("/")[-1].split(".")[0]}_raman')
        continue
    
    raman, photo = split_signal(img)

    stop = timeit.default_timer()
    print('Time: ', stop - start)  

    np.save(f'{file_location2}{f.split("/")[-1].split(".")[0]}_raman', raman.reshape(shape))
    np.save(f'{file_location2}{f.split("/")[-1].split(".")[0]}_photoluminescence', photo.reshape(shape))
    print(f"image: {f} is done.")


file already exist ../data/Raman_Mouse/approximated_curve_fit/Liver_map_150z25_60s_#12_raman
progress = 0.0%, 100 samples time : 0.18869480000012118
progress = 2.6666666666666665%, 100 samples time : 16.7503964
progress = 5.333333333333333%, 100 samples time : 18.122851500000024
progress = 8.0%, 100 samples time : 19.146853299999975
progress = 10.666666666666666%, 100 samples time : 21.939133399999946
progress = 13.333333333333334%, 100 samples time : 27.7493493999998
progress = 16.0%, 100 samples time : 30.35638270000004
progress = 18.666666666666668%, 100 samples time : 28.833296799999744
progress = 21.333333333333332%, 100 samples time : 23.703069600000163
progress = 24.0%, 100 samples time : 20.94863770000029
progress = 26.666666666666668%, 100 samples time : 17.3802483999998
progress = 29.333333333333332%, 100 samples time : 16.900922300000275
progress = 32.0%, 100 samples time : 16.38979389999986
progress = 34.666666666666664%, 100 samples time : 15.697284699999727
progress = 37.

progress = 24.0%, 100 samples time : 22.141697600000043
progress = 26.666666666666668%, 100 samples time : 21.67478410000058
progress = 29.333333333333332%, 100 samples time : 20.16653790000055
progress = 32.0%, 100 samples time : 20.454061499999625
progress = 34.666666666666664%, 100 samples time : 21.116286299999956
progress = 37.333333333333336%, 100 samples time : 20.630618299999696
progress = 40.0%, 100 samples time : 20.209266500000012
progress = 42.666666666666664%, 100 samples time : 20.672431199999664
progress = 45.333333333333336%, 100 samples time : 21.938757900000383
progress = 48.0%, 100 samples time : 20.665334899999834
progress = 50.666666666666664%, 100 samples time : 20.82379630000014
progress = 53.333333333333336%, 100 samples time : 21.405601999999817
progress = 56.0%, 100 samples time : 21.570512399999643
progress = 58.666666666666664%, 100 samples time : 21.19458029999987
progress = 61.333333333333336%, 100 samples time : 20.163918600000216
progress = 64.0%, 100 sa

progress = 50.666666666666664%, 100 samples time : 21.199949100000595
progress = 53.333333333333336%, 100 samples time : 21.374584400000458
progress = 56.0%, 100 samples time : 19.832696600000418
progress = 58.666666666666664%, 100 samples time : 20.280573199999708
progress = 61.333333333333336%, 100 samples time : 20.651326600000175
progress = 64.0%, 100 samples time : 22.28555680000045
progress = 66.66666666666667%, 100 samples time : 23.044613400000344
progress = 69.33333333333333%, 100 samples time : 19.960615900000448
progress = 72.0%, 100 samples time : 21.196673100000226
progress = 74.66666666666667%, 100 samples time : 21.95837869999923
progress = 77.33333333333333%, 100 samples time : 22.276129700000638
progress = 80.0%, 100 samples time : 21.73471530000006
progress = 82.66666666666667%, 100 samples time : 22.006255799999963
progress = 85.33333333333333%, 100 samples time : 21.177984399999332
progress = 88.0%, 100 samples time : 23.52332260000003
progress = 90.66666666666667%,

progress = 77.33333333333333%, 100 samples time : 19.682231699998738
progress = 80.0%, 100 samples time : 20.247858599999745
progress = 82.66666666666667%, 100 samples time : 20.366006500000367
progress = 85.33333333333333%, 100 samples time : 20.892182099998536
progress = 88.0%, 100 samples time : 20.930904900000314
progress = 90.66666666666667%, 100 samples time : 20.641372899999624
progress = 93.33333333333333%, 100 samples time : 20.8944948999997
progress = 96.0%, 100 samples time : 20.044042100000297
progress = 98.66666666666667%, 100 samples time : 19.68497070000012
Time:  705.0491237999995
image: Liver_map_150z25_60s_#03.npy is done.
progress = 0.0%, 100 samples time : 0.11268610000115586
progress = 2.6666666666666665%, 100 samples time : 18.243072700000994
progress = 5.333333333333333%, 100 samples time : 17.00981739999952
progress = 8.0%, 100 samples time : 17.557919499999116
progress = 10.666666666666666%, 100 samples time : 18.725448300001517
progress = 13.333333333333334%, 

progress = 0.0%, 100 samples time : 0.1883562000002712
progress = 2.6666666666666665%, 100 samples time : 15.791818199999398
progress = 5.333333333333333%, 100 samples time : 17.285395200000494
progress = 8.0%, 100 samples time : 16.224848300000303
progress = 10.666666666666666%, 100 samples time : 16.50335730000006
progress = 13.333333333333334%, 100 samples time : 16.336134400000446
progress = 16.0%, 100 samples time : 18.192870999999286
progress = 18.666666666666668%, 100 samples time : 21.613296000001355
progress = 21.333333333333332%, 100 samples time : 18.227128299999094
progress = 24.0%, 100 samples time : 17.028756100000464
progress = 26.666666666666668%, 100 samples time : 17.016517199999726
progress = 29.333333333333332%, 100 samples time : 18.344904800000222
progress = 32.0%, 100 samples time : 20.9249662000002
progress = 34.666666666666664%, 100 samples time : 18.053636099999494
progress = 37.333333333333336%, 100 samples time : 15.693644299999505
progress = 40.0%, 100 samp

...

progress = 85.33333333333333%, 100 samples time : 18.08501729999989
progress = 88.0%, 100 samples time : 18.95958929999688
progress = 90.66666666666667%, 100 samples time : 20.274210600000515
progress = 93.33333333333333%, 100 samples time : 23.09146509999846
progress = 96.0%, 100 samples time : 21.203073000000586
progress = 98.66666666666667%, 100 samples time : 19.860931499999424
Time:  764.125366100001
image: Liver_map_150x25_60s_#7.npy is done.
progress = 0.0%, 100 samples time : 0.3297071999986656
progress = 2.6666666666666665%, 100 samples time : 17.71773899999971
progress = 5.333333333333333%, 100 samples time : 18.100212500001362
progress = 8.0%, 100 samples time : 19.118341100002
progress = 10.666666666666666%, 100 samples time : 17.618145799999184
progress = 13.333333333333334%, 100 samples time : 18.262866700002633
progress = 16.0%, 100 samples time : 17.220405199997913
progress = 18.666666666666668%, 100 samples time : 19.208157699998992
progress = 21.333333333333332%, 100 

progress = 8.0%, 100 samples time : 18.28639530000146
progress = 10.666666666666666%, 100 samples time : 18.549462000002677
progress = 13.333333333333334%, 100 samples time : 18.395437899998797
progress = 16.0%, 100 samples time : 18.334962400000222
progress = 18.666666666666668%, 100 samples time : 19.512531899999885
progress = 21.333333333333332%, 100 samples time : 19.350328100001207
progress = 24.0%, 100 samples time : 18.057950599999458
progress = 26.666666666666668%, 100 samples time : 19.235289599997486
progress = 29.333333333333332%, 100 samples time : 19.4130436000014
progress = 32.0%, 100 samples time : 20.76069379999899
progress = 34.666666666666664%, 100 samples time : 21.55640890000359
progress = 37.333333333333336%, 100 samples time : 22.440803000001324
progress = 40.0%, 100 samples time : 23.775974500000302
progress = 42.666666666666664%, 100 samples time : 23.177115300000878
progress = 45.333333333333336%, 100 samples time : 22.620211199999176
progress = 48.0%, 100 samp

progress = 34.666666666666664%, 100 samples time : 23.617850599999656
progress = 37.333333333333336%, 100 samples time : 25.593858800002636
progress = 40.0%, 100 samples time : 22.25777290000042
progress = 42.666666666666664%, 100 samples time : 21.467814200001158
progress = 45.333333333333336%, 100 samples time : 22.22638250000091
progress = 48.0%, 100 samples time : 20.025608799998736
progress = 50.666666666666664%, 100 samples time : 19.32855749999726
progress = 53.333333333333336%, 100 samples time : 20.185655999997834
progress = 56.0%, 100 samples time : 19.420827900001314
progress = 58.666666666666664%, 100 samples time : 20.289209800001117
progress = 61.333333333333336%, 100 samples time : 23.320850500000233
progress = 64.0%, 100 samples time : 28.351212500001566
progress = 66.66666666666667%, 100 samples time : 26.356537700001354
progress = 69.33333333333333%, 100 samples time : 26.72058830000242
progress = 72.0%, 100 samples time : 26.006836700002168
progress = 74.666666666666

progress = 61.333333333333336%, 100 samples time : 16.81068999999843
progress = 64.0%, 100 samples time : 19.073419100001047
progress = 66.66666666666667%, 100 samples time : 16.959517600000254
progress = 69.33333333333333%, 100 samples time : 16.971608200001356
progress = 72.0%, 100 samples time : 16.18778790000215
progress = 74.66666666666667%, 100 samples time : 15.800459000005503
progress = 77.33333333333333%, 100 samples time : 16.210789099997783
progress = 80.0%, 100 samples time : 14.578872899997805
progress = 82.66666666666667%, 100 samples time : 15.223774799997045
progress = 85.33333333333333%, 100 samples time : 16.913181500000064
progress = 88.0%, 100 samples time : 16.67221260000224
progress = 90.66666666666667%, 100 samples time : 16.14664499999344
progress = 93.33333333333333%, 100 samples time : 15.237953100004233
progress = 96.0%, 100 samples time : 16.384882300000754
progress = 98.66666666666667%, 100 samples time : 16.927617100001953
Time:  618.7749906999961
image: K

progress = 88.0%, 100 samples time : 17.722328800002288
progress = 90.66666666666667%, 100 samples time : 17.615569299996423
progress = 93.33333333333333%, 100 samples time : 15.486246000000392
progress = 96.0%, 100 samples time : 16.0071597999995
progress = 98.66666666666667%, 100 samples time : 16.872473499999614
Time:  610.9160233000002
image: Liver_map_150z25_60s_#4.npy is done.
progress = 0.0%, 100 samples time : 0.23702220000268426
progress = 2.6666666666666665%, 100 samples time : 13.198897199996281
progress = 5.333333333333333%, 100 samples time : 14.236922700001742
progress = 8.0%, 100 samples time : 13.656019100002595
progress = 10.666666666666666%, 100 samples time : 12.78856960000121
progress = 13.333333333333334%, 100 samples time : 13.473738100001356
progress = 16.0%, 100 samples time : 14.796425300002738
progress = 18.666666666666668%, 100 samples time : 14.215988500000094
progress = 21.333333333333332%, 100 samples time : 14.128406100004213
progress = 24.0%, 100 samples